# 🔬 Validación Matemática Completa - Fórmula Explícita Funcional

**Autor:** José Manuel Mota Burruezo (JMMB Ψ✧)  
**Objetivo:** Validación completa de la fórmula explícita funcional  
**Método:** Comparación lado Arquimediano vs lado aritmético

---

Este notebook implementa la validación matemática rigurosa de la fórmula explícita funcional,
comparando el lado Arquimediano A_∞(f) con la suma de primos y ceros de zeta.

⚠️ **Advertencia:** Este notebook usa parámetros altos para validación científica completa.
Para ejecución rápida en CI, usar `validation_quick.ipynb`.


In [ ]:
# 📦 Instalación de dependencias matemáticas
!pip install mpmath sympy numpy scipy matplotlib --quiet

print("✅ Dependencias matemáticas instaladas")

In [ ]:
# 🔧 Importar bibliotecas
import mpmath as mp
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from scipy.special import gamma
import time
import warnings
warnings.filterwarnings('ignore')

# Configurar precisión alta para mpmath
mp.dps = 50  # 50 dígitos decimales de precisión

print("🔧 Bibliotecas importadas correctamente")
print(f"📊 Precisión mpmath: {mp.dps} dígitos decimales")

In [ ]:
# 🎯 Parámetros de validación completa (CIENTÍFICOS - pueden causar timeout en CI)
P = 1000    # Número de primos a usar
K = 50      # Parámetro K para funciones test
NΞ = 2000   # Número de ceros de zeta
T = 50      # Altura límite para cálculos

# Configuración
lim = 10.0  # Límite de integración
σ0 = 1.5    # Línea crítica desplazada

print(f"🎯 Parámetros de validación:")
print(f"   P (primos): {P}")
print(f"   K (test): {K}")
print(f"   NΞ (ceros): {NΞ}")
print(f"   T (altura): {T}")
print(f"   lim: {lim}")
print(f"   σ0: {σ0}")

In [ ]:
# 🔢 Generación de primos usando criba de Eratóstenes optimizada
def sieve_of_eratosthenes(limit):
    """Criba de Eratóstenes optimizada para generar primos"""
    sieve = [True] * (limit + 1)
    sieve[0] = sieve[1] = False
    
    for i in range(2, int(limit**0.5) + 1):
        if sieve[i]:
            for j in range(i*i, limit + 1, i):
                sieve[j] = False
    
    return [i for i in range(2, limit + 1) if sieve[i]]

# Generar los primeros P primos
print(f"🔢 Generando los primeros {P} primos...")
start_time = time.time()

# Estimación del límite superior para obtener P primos
if P < 6:
    limit = 15
else:
    limit = int(P * (np.log(P) + np.log(np.log(P))))

all_primes = sieve_of_eratosthenes(limit)
while len(all_primes) < P:
    limit *= 2
    all_primes = sieve_of_eratosthenes(limit)

primes = all_primes[:P]
generation_time = time.time() - start_time

print(f"✅ {P} primos generados en {generation_time:.2f} segundos")
print(f"   Rango: {primes[0]} - {primes[-1]}")
print(f"   Primeros 10: {primes[:10]}")
print(f"   Últimos 10: {primes[-10:]}")

In [ ]:
# ζ Cálculo de ceros no triviales de la función zeta de Riemann
def calculate_zeta_zeros(N):
    """Calcular los primeros N ceros no triviales de zeta usando mpmath"""
    zeros = []
    print(f"🔍 Calculando {N} ceros de ζ(s)...")
    
    for n in range(1, N + 1):
        if n % 100 == 0:
            print(f"   Progreso: {n}/{N} ceros calculados...")
        
        # Calcular el n-ésimo cero usando mpmath
        zero_imag = mp.zetazero(n)
        zeros.append(complex(0.5, float(zero_imag)))
    
    return zeros

print(f"ζ Calculando los primeros {NΞ} ceros de zeta...")
start_time = time.time()

zeta_zeros = calculate_zeta_zeros(NΞ)
zeros_time = time.time() - start_time

print(f"✅ {NΞ} ceros calculados en {zeros_time:.2f} segundos")
print(f"   Primeros 5 ceros (partes imaginarias): {[z.imag for z in zeta_zeros[:5]]}")
print(f"   Últimos 5 ceros (partes imaginarias): {[z.imag for z in zeta_zeros[-5:]]}")

In [ ]:
# 🧮 Funciones test para validación
def gaussian_test_function(u, k=1.0):
    """Función test Gaussiana: f(u) = exp(-k*u^2)"""
    return np.exp(-k * u**2)

def compact_test_function(u, k=1.0):
    """Función test compacta con soporte [-k, k]"""
    return np.where(np.abs(u) <= k, np.exp(-1/(k**2 - u**2)), 0.0)

# Seleccionar función test
def test_function(u):
    """Función test principal - Gaussiana par"""
    return gaussian_test_function(u, k=1.0/K)

print(f"🧮 Funciones test definidas")
print(f"   Usando Gaussiana con k = 1/{K} = {1.0/K}")

# Verificar que la función es par
test_points = np.linspace(-2, 2, 100)
parity_check = np.allclose(test_function(test_points), test_function(-test_points))
print(f"✅ Verificación de paridad: {'PASADA' if parity_check else 'FALLIDA'}")

In [ ]:
# 🏛️ Cálculo del lado Arquimediano A_∞(f)
def archimedean_side(f_func, lim_val):
    """Calcular el lado Arquimediano A_∞(f) usando integración numérica de alta precisión"""
    print(f"🏛️ Calculando lado Arquimediano...")
    start_time = time.time()
    
    # Integral: ∫_{-lim}^{lim} f(u) * [1/2 * ψ(|u|/2 + 1) - 1/2 * log(π)] du
    def integrand(u):
        u_abs = abs(float(u))
        if u_abs < 1e-10:
            return 0.0
        
        # Función digamma ψ(x) usando mpmath para alta precisión
        psi_val = float(mp.psi(u_abs/2 + 1))
        kernel = 0.5 * psi_val - 0.5 * np.log(np.pi)
        
        return f_func(u) * kernel
    
    # Integración adaptativa usando mpmath
    result = float(mp.quad(integrand, [-lim_val, lim_val], maxdegree=10))
    
    calc_time = time.time() - start_time
    print(f"✅ Lado Arquimediano calculado en {calc_time:.2f} segundos")
    print(f"   A_∞(f) = {result}")
    
    return result

A_infinity = archimedean_side(test_function, lim)

In [ ]:
# 🔢 Cálculo de la suma sobre primos
def prime_sum(f_func, primes_list, lim_val):
    """Calcular suma sobre primos: Σ_p log(p) * ∫ f(u) p^{-iu} du"""
    print(f"🔢 Calculando suma sobre {len(primes_list)} primos...")
    start_time = time.time()
    
    total_sum = 0.0
    
    for i, p in enumerate(primes_list):
        if i % 100 == 0 and i > 0:
            print(f"   Progreso primos: {i}/{len(primes_list)}")
        
        log_p = np.log(p)
        
        # Integral: ∫_{-lim}^{lim} f(u) * p^{-iu} du
        def integrand(u):
            return f_func(u) * np.exp(-1j * u * np.log(p))
        
        # Integración compleja usando mpmath
        integral_result = complex(mp.quad(integrand, [-lim_val, lim_val], maxdegree=8))
        
        # Solo tomar parte real (la función f es par, así que la parte imaginaria debería ser ~0)
        contribution = log_p * integral_result.real
        total_sum += contribution
    
    calc_time = time.time() - start_time
    print(f"✅ Suma sobre primos calculada en {calc_time:.2f} segundos")
    print(f"   Σ_p = {total_sum}")
    
    return total_sum

prime_contribution = prime_sum(test_function, primes, lim)

In [ ]:
# ζ Cálculo de la suma sobre ceros de zeta
def zero_sum(f_func, zeros_list, lim_val, sigma0):
    """Calcular suma sobre ceros: Σ_ρ ∫ f(u) * (σ0 + iγ)^{-iu} du"""
    print(f"ζ Calculando suma sobre {len(zeros_list)} ceros...")
    start_time = time.time()
    
    total_sum = 0.0
    
    for i, zero in enumerate(zeros_list):
        if i % 200 == 0 and i > 0:
            print(f"   Progreso ceros: {i}/{len(zeros_list)}")
        
        # zero = 0.5 + i*gamma, pero usamos σ0 + i*gamma
        rho = complex(sigma0, zero.imag)
        
        # Integral: ∫_{-lim}^{lim} f(u) * ρ^{-iu} du
        def integrand(u):
            # ρ^{-iu} = exp(-iu * log(ρ))
            log_rho = np.log(rho)
            return f_func(u) * np.exp(-1j * u * log_rho)
        
        # Integración compleja
        integral_result = complex(mp.quad(integrand, [-lim_val, lim_val], maxdegree=8))
        
        # Solo parte real
        contribution = integral_result.real
        total_sum += contribution
    
    calc_time = time.time() - start_time
    print(f"✅ Suma sobre ceros calculada en {calc_time:.2f} segundos")
    print(f"   Σ_ρ = {total_sum}")
    
    return total_sum

zero_contribution = zero_sum(test_function, zeta_zeros, lim, σ0)

In [ ]:
# 📊 Comparación y validación final
arithmetic_side = prime_contribution + zero_contribution

print(f"\n📊 RESULTADOS DE VALIDACIÓN:")
print(f"=" * 50)
print(f"🏛️ Lado Arquimediano: A_∞(f) = {A_infinity:.10f}")
print(f"🔢 Suma de primos:             = {prime_contribution:.10f}")
print(f"ζ  Suma de ceros:              = {zero_contribution:.10f}")
print(f"📈 Lado aritmético total:      = {arithmetic_side:.10f}")
print(f"=" * 50)

# Cálculo de errores
absolute_error = abs(A_infinity - arithmetic_side)
relative_error = absolute_error / abs(A_infinity) if A_infinity != 0 else float('inf')

print(f"❌ Error absoluto: {absolute_error:.10f}")
print(f"📊 Error relativo: {relative_error:.10f}")

# Criterios de validación
tolerance = 1e-6
validation_passed = absolute_error < tolerance

print(f"\n🎯 VALIDACIÓN:")
print(f"   Tolerancia objetivo: {tolerance}")
print(f"   Estado: {'✅ PASADA' if validation_passed else '❌ FALLIDA'}")

if not validation_passed:
    print(f"\n⚠️ POSIBLES CAUSAS DEL ERROR GRANDE:")
    print(f"   - Número insuficiente de ceros (NΞ={NΞ})")
    print(f"   - Número insuficiente de primos (P={P})")
    print(f"   - Problema de normalización (σ0={σ0})")
    print(f"   - Límite de integración demasiado pequeño (lim={lim})")
    print(f"\n💡 RECOMENDACIONES:")
    print(f"   - Aumentar NΞ progresivamente: 2000 → 5000 → 10000")
    print(f"   - Verificar σ0 = 1.5 vs σ0 = 2.0")
    print(f"   - Probar función test más simple (Gaussiana pura)")

print(f"\n📝 Parámetros utilizados:")
print(f"   P={P}, K={K}, NΞ={NΞ}, T={T}")
print(f"   lim={lim}, σ0={σ0}")